# Similarity Ensemble

In [70]:
cd recsys_challenge_2017/

/Users/emanueleghelfi/Development/RS/recsys_challenge_2017


In [71]:
from src.utils.loader import *
from scipy.sparse import *
import numpy as np
from src.utils.feature_weighting import *
from src.utils.matrix_utils import compute_cosine, top_k_filtering, yadistance
from src.utils.BaseRecommender import BaseRecommender


class ContentBasedFiltering(BaseRecommender):

    """
    Good conf: tag aggr 3,10; tfidf l1 norm over all matrix
    MAP@5  0.11772497678137457 with 10 shrinkage,
                                    100 k_filtering and other as before
    MAP@5  0.12039006297936491 urm weight 0.7
    MAP@5  0.12109109578826009 without playcount and duration

    Current best:
    CBF (album 1.0, artists 1.0, no duration/playcount)
        + URM 0.8
        + TOP-55 (TFIDF (tags 1.0))
        MAP@5 0.11897304011860126
        Public leaderboard: 0.09616


    """

    def __init__(self, shrinkage=10, k_filtering=100):
        # final matrix of predictions
        self.R_hat = None

        # for keeping reference between playlist and row index
        self.pl_id_list = []
        # for keeping reference between tracks and column index
        self.tr_id_list = []

        self.shrinkage = shrinkage
        self.k_filtering = k_filtering

    def fit(self, urm, target_playlist, target_tracks, dataset, topK_tag=55):
        """
        urm: user rating matrix
        target playlist is a list of playlist id
        target_tracks is a list of track id
        shrinkage: shrinkage factor for significance weighting
        S = ICM' ICM
        R = URM S
        In between eliminate useless row of URM and useless cols of S
        """
        # initialization

        self.pl_id_list = list(target_playlist)
        self.tr_id_list = list(target_tracks)
        self.dataset = dataset
        S = None
        urm = urm.tocsr()
        print("CBF started")
        # get ICM from dataset, assume it already cleaned
        icm = dataset.build_icm()

        # Build the tag matrix, apply TFIDF
        print("Build tags matrix and apply TFIDF...")
        icm_tag = dataset.build_tags_matrix()
        #tag_weight = np.array(urm.dot(icm_tag.transpose()).sum(axis=0)).squeeze()
        #print(tag_weight[tag_weight>5].shape)
        #icm_tag = icm_tag[tag_weight>5].multiply(tag_weight[tag_weight>5])
        tags = applyTFIDF(icm_tag, norm='l1')
        #tags = icm_tag[tag_weight>5]
        # Before stacking tags with the rest of the ICM, we keep only
        # the top K tags for each item. This way we try to reduce the
        # natural noise added by such sparse features.
        tags = top_k_filtering(tags.transpose(), topK=topK_tag).transpose()
        #tags.data = tags.data / np.max(tags.data)
        #newvalue= (max'-min')/(max-min)*(value-max)+max'

        # User augmented UCM
        # print("Building User augmented ICM")
        # ucm = dataset.build_ucm()
        # ua_icm = user_augmented_icm(urm, ucm)
        # ua_icm = top_k_filtering(ua_icm.transpose(), topK=55).transpose()

        # stack all
        icm = vstack([icm, tags, urm * 1], format='csr')
        # icm = vstack([icm, tags, applyTFIDF(urm)], format='csr')

        S = compute_cosine(icm.transpose()[[dataset.get_track_index_from_id(x)
                                            for x in self.tr_id_list]],
                       icm,
                       k_filtering=self.k_filtering,
                       shrinkage=self.shrinkage,
                       n_threads=4,
                       chunksize=1000)
        s_norm = S.sum(axis=1)

        # Normalize S
        S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
        print("Similarity matrix ready!")

        # Keep only the target playlists in the URM
        urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                           for x in self.pl_id_list]]
        self.S = S.transpose()

        # Compute ratings
        R_hat = urm_cleaned.dot(S.transpose().tocsc()).tocsr()
        print("R_hat done")

        # Remove the entries in R_hat that are already present in the URM
        urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                      for x in self.tr_id_list]]
        R_hat[urm_cleaned.nonzero()] = 0
        R_hat.eliminate_zeros()

        print("Shape of final matrix: ", R_hat.shape)
        self.R_hat = R_hat

    def getW(self):
        """
        Returns the similary matrix with dimensions I x I
        S is IxT
        """
        return self.S.tocsr()

    def predict(self, at=5):
        """
        returns a dictionary of
        'pl_id': ['tr_1', 'tr_at'] for each playlist in target playlist
        """
        recs = {}
        for i in range(0, self.R_hat.shape[0]):
            pl_id = self.pl_id_list[i]
            pl_row = self.R_hat.data[self.R_hat.indptr[i]:
                                     self.R_hat.indptr[i + 1]]
            # get top 5 indeces. argsort, flip and get first at-1 items
            sorted_row_idx = np.flip(pl_row.argsort(), axis=0)[0:at]
            track_cols = [self.R_hat.indices[self.R_hat.indptr[i] + x]
                          for x in sorted_row_idx]
            tracks_ids = [self.tr_id_list[x] for x in track_cols]
            recs[pl_id] = tracks_ids
        return recs

    def getR_hat(self):
        return self.R_hat

    def get_model(self):
        """
        Returns the complete R_hat
        """
        return self.R_hat.copy()


def applyTFIDF(matrix, norm='l1'):
    from sklearn.feature_extraction.text import TfidfTransformer
    transformer = TfidfTransformer(norm=norm, use_idf=True,
                                   smooth_idf=True, sublinear_tf=False)
    tfidf = transformer.fit_transform(matrix.transpose())
    return tfidf.transpose()


def produceCsv():
    # export csv
    dataset = Dataset(load_tags=True,
                      filter_tag=False,
                      weight_tag=False)
    dataset.set_track_attr_weights_2(1.0, 1.0, 0.0, 0.0, 0.0,
                                     1.0, 1.0, 0.0, 0.0)
    cbf_exporter = ContentBasedFiltering()
    urm = dataset.build_train_matrix()
    tg_playlist = list(dataset.target_playlists.keys())
    tg_tracks = list(dataset.target_tracks.keys())
    # Train the model with the best shrinkage found in cross-validation
    cbf_exporter.fit(urm,
                     tg_playlist,
                     tg_tracks,
                     dataset)
    recs = cbf_exporter.predict()
    with open('submission_cbf.csv', mode='w', newline='') as out:
        fieldnames = ['playlist_id', 'track_ids']
        writer = csv.DictWriter(out, fieldnames=fieldnames, delimiter=',')
        writer.writeheader()
        for k in tg_playlist:
            track_ids = ''
            for r in recs[k]:
                track_ids = track_ids + r + ' '
            writer.writerow({'playlist_id': k,
                             'track_ids': track_ids[:-1]})


def evaluateMap():
    from src.utils.evaluator import Evaluator
    dataset = Dataset(load_tags=True,
                      filter_tag=False,
                      weight_tag=False)
    dataset.set_track_attr_weights_2(1.0, 1.0, 0.0, 0.0, 0.0,
                                     1.0, 1.0, 0.0, 0.0)
    # seed = 0xcafebabe
    # print("Evaluating with initial seed: {}".format(seed))
    ev = Evaluator(seed=False)
    ev.cross_validation(5, dataset.train_final.copy())
    cbf = ContentBasedFiltering()
    for i in range(0, 5):
        urm, tg_tracks, tg_playlist = ev.get_fold(dataset)
        cbf.fit(urm,
                list(tg_playlist),
                list(tg_tracks),
                dataset)
        recs = cbf.predict()
        ev.evaluate_fold(recs)

    map_at_five = ev.get_mean_map()
    print("MAP@5 ", map_at_five)


def crossValidation():
    from src.utils.evaluator import Evaluator
    pass

# Item Based Similarity

In [72]:
def ibs(urm, dataset):
    urm_sim = applyTFIDF(urm)
    #urm_sim = top_k_filtering(urm_sim.transpose()).transpose()
    S = compute_cosine(urm_sim.transpose()[[dataset.get_track_index_from_id(x)
                                    for x in tg_tracks]],
                       urm_sim,
                       k_filtering=100,
                       shrinkage=10,
                       n_threads=4,
                       chunksize=1000)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

# User Based Similarity

In [73]:
def ubs(urm, ucm, dataset):
    ucm = dataset.build_ucm()
    iucm = ucm.dot(urm)
    iucm = applyTFIDF(iucm)
    iucm = top_k_filtering(iucm.transpose(), 100).transpose()
    S = compute_cosine(iucm.transpose()[[dataset.get_track_index_from_id(x)
                                                for x in tg_tracks]],
                           iucm,
                           k_filtering=100,
                           shrinkage=10,
                           n_threads=4,
                           chunksize=1000)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    print("S done")
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

# Only Content Based

In [74]:
def icm_sim(icm, dataset):
    icm_tag = dataset.build_tags_matrix()
    #tag_weight = np.array(urm.dot(icm_tag.transpose()).sum(axis=0)).squeeze()
    #print(tag_weight[tag_weight>5].shape)
    #icm_tag = icm_tag[tag_weight>5].multiply(tag_weight[tag_weight>5])
    tags = applyTFIDF(icm_tag, norm='l1')
    #tags = icm_tag[tag_weight>5]
    # Before stacking tags with the rest of the ICM, we keep only
    # the top K tags for each item. This way we try to reduce the
    # natural noise added by such sparse features.
    tags = top_k_filtering(tags.transpose(), topK=55).transpose()
    #tags.data = tags.data / np.max(tags.data)
    #newvalue= (max'-min')/(max-min)*(value-max)+max'

    # User augmented UCM
    # print("Building User augmented ICM")
    # ucm = dataset.build_ucm()
    # ua_icm = user_augmented_icm(urm, ucm)
    # ua_icm = top_k_filtering(ua_icm.transpose(), topK=55).transpose()

    # stack all
    icm = vstack([icm, tags*0.4], format='csr')
    #icm = applyTFIDF(icm)
    #icm = top_k_filtering(icm.transpose(), topK=200).transpose()
    # icm = vstack([icm, tags, applyTFIDF(urm)], format='csr')

    S = compute_cosine(icm.transpose()[[dataset.get_track_index_from_id(x)
                                        for x in tg_tracks]],
                   icm,
                   k_filtering=100,
                   shrinkage=10,
                   n_threads=4,
                   chunksize=1000)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    print("S done")
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

In [75]:
def icm_sim2(icm, dataset):

    S = compute_cosine(icm.transpose()[[dataset.get_track_index_from_id(x)
                                        for x in tg_tracks]],
                   icm,
                   k_filtering=100,
                   shrinkage=10,
                   n_threads=4,
                   chunksize=1000)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    print("S done")
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

# MF-Based Similarity

In [92]:
from src.CBF.CBF_MF import ContentBasedFiltering as CBF
from src.MF.MF_BPR.MF_BPR import MF_BPR
cbf = CBF()
    
cbf.fit(urm, tg_playlist,
        tg_tracks,
        dataset)

# get R_hat
R_hat_aug = cbf.getR_hat()
print(R_hat_aug.nnz)

CBF started
Build tags matrix and apply TFIDF...
Running 4 workers...
[ 13171 ] Building cosine similarity matrix for [0, 1000)...
[ 13172 ] Building cosine similarity matrix for [25000, 26000)...
[ 13173 ] Building cosine similarity matrix for [50000, 51000)...
[ 13171 ] Building cosine similarity matrix for [1000, 2000)...
[ 13174 ] Building cosine similarity matrix for [75000, 76000)...
[ 13172 ] Building cosine similarity matrix for [26000, 27000)...
[ 13171 ] Building cosine similarity matrix for [2000, 3000)...
[ 13173 ] Building cosine similarity matrix for [51000, 52000)...
[ 13172 ] Building cosine similarity matrix for [27000, 28000)...
[ 13174 ] Building cosine similarity matrix for [76000, 77000)...
[ 13171 ] Building cosine similarity matrix for [3000, 4000)...
[ 13173 ] Building cosine similarity matrix for [52000, 53000)...
[ 13172 ] Building cosine similarity matrix for [28000, 29000)...
[ 13171 ] Building cosine similarity matrix for [4000, 5000)...
[ 13173 ] Building 

/Users/emanueleghelfi/Development/RS/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Shape of final matrix:  (57561, 100000)
1246601


In [93]:
mf = MF_BPR()

In [94]:
print("Started learning MF")
# MAP@5: 0.08256503053607782 with 500 factors after 10 epochs
# MAP@5: 0.08594586443489391 with 500 factors afetr 4 epochs no_components=500, epoch_multiplier=2, l_rate=1e-2
mf.fit(R_hat_aug, dataset, list(tg_playlist), list(tg_tracks), n_epochs=1, no_components=500, epoch_multiplier=6, l_rate=1e-2, use_icm=False)
#recs = mf.predict_dot_custom(urm)
#ev.evaluate_fold(recs)

# MAP@5: 0.09407901681369218 with neighborhood
# MAP@5: 0.  with neighborhood after 4 epochs
# MAP@5 after 2 epoch with epoch mult = 3 (6 epochs)  0.09326999924602265 with 500 factors,
# Urm augmented by 5 each user by cbf.
recs = mf.predict_knn_custom(urm)
ev.evaluate_fold(recs)

S_mf = mf.S

Started learning MF
Processed 5000000 ( 66.85% ) in 493.81 seconds. Sample per second: 10125
Processed 7479606 ( 100.00% ) in 250.30 seconds. Sample per second: 10063
Training finished
Computing dot product for chunk [0, 1000)...
Computing dot product for chunk [1000, 2000)...
Computing dot product for chunk [2000, 3000)...
Computing dot product for chunk [3000, 4000)...
Computing dot product for chunk [4000, 5000)...
Computing dot product for chunk [5000, 6000)...
Computing dot product for chunk [6000, 7000)...
Computing dot product for chunk [7000, 8000)...
Computing dot product for chunk [8000, 9000)...
Computing dot product for chunk [9000, 10000)...
Computing dot product for chunk [10000, 11000)...
Computing dot product for chunk [11000, 12000)...
Computing dot product for chunk [12000, 13000)...
Computing dot product for chunk [13000, 14000)...
Computing dot product for chunk [14000, 15000)...
Computing dot product for chunk [15000, 16000)...
Computing dot product for chunk [1600

# SLIM Similarity

In [9]:
from src.ML.BPRSLIM import BPRSLIM
import os


dataset.set_track_attr_weights_2(1, 1, 0, 0, 0, 0, 0, 0, 0)
#ds.set_track_attr_weights(1, 1, 0, 0, 1)
print('Building the ICM...')
icm = dataset.build_icm()

icm_tag = dataset.build_tags_matrix()

tags = applyTFIDF(icm_tag, norm='l1')

# Before stacking tags with the rest of the ICM, we keep only
# the top K tags for each item. This way we try to reduce the
# natural noise added by such sparse features.
tags = top_k_filtering(tags.transpose(), topK=55).transpose()
tags.data = np.ones_like(tags.data)

# stack all
icm = vstack([icm, tags], format='csr')

recommender = BPRSLIM(epochs=50,
                      epochMultiplier=1.0,
                      sgd_mode='rmsprop',
                      learning_rate=5e-02,
                      topK=300,
                      urmSamplingChances=1 / 5,
                      icmSamplingChances=4 / 5)
recommender.set_evaluation_every(1, ev)
recommender.fit(urm.tocsr(),
                icm.tocsr(),
                tg_playlist,
                tg_tracks,
                dataset)

S_bprslim = recommender.getParameters()
S_bprslim = S_bprslim[:,[dataset.get_track_index_from_id(x) for x in tg_tracks]]

# keep only 100 elements
S_bprslim = top_k_filtering(S_bprslim.transpose(), 100).transpose()

# normalize
s_norm = S_bprslim.sum(axis=0)
s_norm[s_norm==0] = 1
print("S done")

S_bprslim = S_bprslim.multiply(csr_matrix(np.reciprocal(s_norm)))


Building the ICM...
Running fit process.
Processed 500000 ( 29.85% ) in 23.64 seconds. Sample per second: 21153
Processed 1000000 ( 59.70% ) in 8.64 seconds. Sample per second: 31606
Processed 1500000 ( 89.54% ) in 7.28 seconds. Sample per second: 39189
Processed 1675142 ( 100.00% ) in 2.72 seconds. Sample per second: 41140
Return S matrix to python caller...
Epoch 0 of 50 complete in 0.87 minutes
Processed 500000 ( 29.85% ) in 7.59 seconds. Sample per second: 65876
Processed 1000000 ( 59.70% ) in 6.32 seconds. Sample per second: 75075
Processed 1500000 ( 89.54% ) in 5.85 seconds. Sample per second: 79592
Processed 1675142 ( 100.00% ) in 2.75 seconds. Sample per second: 80739
Return S matrix to python caller...
Evaluating recommendations
R_hat evaluated...


/Users/emanueleghelfi/Development/RS/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


MAP@5: 0.06886375631455911
Epoch 1 of 50 complete in 0.90 minutes
Processed 500000 ( 29.85% ) in 6.26 seconds. Sample per second: 79876
Processed 1000000 ( 59.70% ) in 6.28 seconds. Sample per second: 81458
Processed 1500000 ( 89.54% ) in 6.10 seconds. Sample per second: 82880
Processed 1675142 ( 100.00% ) in 2.06 seconds. Sample per second: 83498
Return S matrix to python caller...
Evaluating recommendations
R_hat evaluated...
MAP@5: 0.07091306642539365
Epoch 2 of 50 complete in 0.94 minutes
Processed 500000 ( 29.85% ) in 6.28 seconds. Sample per second: 79597
Processed 1000000 ( 59.70% ) in 5.78 seconds. Sample per second: 84867
Processed 1500000 ( 89.54% ) in 6.35 seconds. Sample per second: 86449
Processed 1675142 ( 100.00% ) in 2.20 seconds. Sample per second: 87261
Return S matrix to python caller...
Evaluating recommendations
R_hat evaluated...
MAP@5: 0.07018246248963242
Epoch 3 of 50 complete in 1.03 minutes
Processed 500000 ( 29.85% ) in 6.12 seconds. Sample per second: 81740


KeyboardInterrupt: 

# Initialization

In [91]:
from src.utils.evaluator import Evaluator
from src.utils.loader import *
print("started")
print("hey")
dataset = Dataset(load_tags=True,
                  filter_tag=False,
                  weight_tag=False)
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
# seed = 0xcafebabe
# print("Evaluating with initial seed: {}".format(seed))
ev = Evaluator(seed=False)
ev.cross_validation(5, dataset.train_final.copy())
urm, tg_tracks, tg_playlist = ev.get_fold(dataset)
print("Done")

started
hey
File found, retrieving urm from it.
Load from file takes 0.21 seconds
Done


In [76]:
print("Start exporting")
dataset = Dataset(load_tags=True,
                  filter_tag=False,
                  weight_tag=False)
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
# seed = 0xcafebabe
# print("Evaluating with initial seed: {}".format(seed))
urm = dataset.build_train_matrix()
tg_playlist = list(dataset.target_playlists.keys())
tg_tracks = list(dataset.target_tracks.keys())

Start exporting
File found, retrieving urm from it.
Load from file takes 0.21 seconds


# Ensemble

In [95]:
print("ok")
cbf = ContentBasedFiltering()
cbf.fit(urm,
        list(tg_playlist),
        list(tg_tracks),
        dataset, 
        topK_tag=55)
recs = cbf.predict()
ev.evaluate_fold(recs)
S_cbf_full = cbf.S

ok
CBF started
Build tags matrix and apply TFIDF...
Running 4 workers...
[ 13258 ] Building cosine similarity matrix for [0, 1000)...
[ 13259 ] Building cosine similarity matrix for [4277, 5277)...
[ 13260 ] Building cosine similarity matrix for [8554, 9554)...
[ 13258 ] Building cosine similarity matrix for [1000, 2000)...
[ 13261 ] Building cosine similarity matrix for [12831, 13831)...
[ 13259 ] Building cosine similarity matrix for [5277, 6277)...
[ 13260 ] Building cosine similarity matrix for [9554, 10554)...
[ 13258 ] Building cosine similarity matrix for [2000, 3000)...
[ 13261 ] Building cosine similarity matrix for [13831, 14831)...
[ 13259 ] Building cosine similarity matrix for [6277, 7277)...
[ 13260 ] Building cosine similarity matrix for [10554, 11554)...
[ 13258 ] Building cosine similarity matrix for [3000, 4000)...
[ 13261 ] Building cosine similarity matrix for [14831, 15831)...
[ 13259 ] Building cosine similarity matrix for [7277, 8277)...
[ 13260 ] Building cosine

In [96]:
dataset.set_playlist_attr_weights(1, 1, 1, 0, 0)
ucm = dataset.build_ucm()
S_user = ubs(urm, ucm, dataset)

Running 4 workers...
[ 13267 ] Building cosine similarity matrix for [0, 1000)...
[ 13268 ] Building cosine similarity matrix for [4277, 5277)...
[ 13269 ] Building cosine similarity matrix for [8554, 9554)...
[ 13270 ] Building cosine similarity matrix for [12831, 13831)...
[ 13267 ] Building cosine similarity matrix for [1000, 2000)...
[ 13268 ] Building cosine similarity matrix for [5277, 6277)...
[ 13269 ] Building cosine similarity matrix for [9554, 10554)...
[ 13270 ] Building cosine similarity matrix for [13831, 14831)...
[ 13267 ] Building cosine similarity matrix for [2000, 3000)...
[ 13268 ] Building cosine similarity matrix for [6277, 7277)...
[ 13269 ] Building cosine similarity matrix for [10554, 11554)...
[ 13270 ] Building cosine similarity matrix for [14831, 15831)...
[ 13267 ] Building cosine similarity matrix for [3000, 4000)...
[ 13268 ] Building cosine similarity matrix for [7277, 8277)...
[ 13269 ] Building cosine similarity matrix for [11554, 12554)...
[ 13270 ] B

In [97]:
S_urm = ibs(urm, dataset)

Running 4 workers...
[ 13306 ] Building cosine similarity matrix for [0, 1000)...
[ 13307 ] Building cosine similarity matrix for [4277, 5277)...
[ 13308 ] Building cosine similarity matrix for [8554, 9554)...
[ 13306 ] Building cosine similarity matrix for [1000, 2000)...
[ 13309 ] Building cosine similarity matrix for [12831, 13831)...
[ 13306 ] Building cosine similarity matrix for [2000, 3000)...
[ 13307 ] Building cosine similarity matrix for [5277, 6277)...
[ 13308 ] Building cosine similarity matrix for [9554, 10554)...
[ 13307 ] Building cosine similarity matrix for [6277, 7277)...
[ 13306 ] Building cosine similarity matrix for [3000, 4000)...
[ 13308 ] Building cosine similarity matrix for [10554, 11554)...
[ 13307 ] Building cosine similarity matrix for [7277, 8277)...
[ 13306 ] Building cosine similarity matrix for [4000, 4277)...
[ 13309 ] Building cosine similarity matrix for [13831, 14831)...
[ 13308 ] Building cosine similarity matrix for [11554, 12554)...
[ 13307 ] Bui

In [98]:
# 1.5, 1.6, 0.1, 0.2, 0.0, 1.0, 0, 0.0, 0.0  MAP 0.102
# 1.5, 1.6, 0.1, 0.2, 0.0, 0.0, 1.0, 0, 0.0 MAP 0.1033928975344946

dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 0.0, 1.0, 0, 0.0)
S_icm = icm_sim(dataset.build_icm(), dataset)

[ 13337 ] Building cosine similarity matrix for [0, 1000)...
Running 4 workers...
[ 13338 ] Building cosine similarity matrix for [4277, 5277)...
[ 13339 ] Building cosine similarity matrix for [8554, 9554)...
[ 13340 ] Building cosine similarity matrix for [12831, 13831)...
[ 13337 ] Building cosine similarity matrix for [1000, 2000)...
[ 13338 ] Building cosine similarity matrix for [5277, 6277)...
[ 13340 ] Building cosine similarity matrix for [13831, 14831)...
[ 13339 ] Building cosine similarity matrix for [9554, 10554)...
[ 13337 ] Building cosine similarity matrix for [2000, 3000)...
[ 13340 ] Building cosine similarity matrix for [14831, 15831)...
[ 13338 ] Building cosine similarity matrix for [6277, 7277)...
[ 13339 ] Building cosine similarity matrix for [10554, 11554)...
[ 13337 ] Building cosine similarity matrix for [3000, 4000)...
[ 13340 ] Building cosine similarity matrix for [15831, 16831)...
[ 13338 ] Building cosine similarity matrix for [7277, 8277)...
[ 13339 ] B

In [99]:
# 0.080286755758088982, 0.18553973793824477, 0.0075749795138103666, 0.011466764550627563, 0.010195035817612831, 0.049304432078911345

dataset.set_track_attr_weights_2(0.080286755758088982, 0.18553973793824477, 0.0075749795138103666, 0.011466764550627563, 0.010195035817612831,
                                 0.0, 0.049304432078911345, 0, 0.0)
S_icm2 = icm_sim2(dataset.build_icm(), dataset)

[ 13342 ] Building cosine similarity matrix for [0, 1000)...
Running 4 workers...
[ 13343 ] Building cosine similarity matrix for [4277, 5277)...
[ 13344 ] Building cosine similarity matrix for [8554, 9554)...
[ 13345 ] Building cosine similarity matrix for [12831, 13831)...
[ 13342 ] Building cosine similarity matrix for [1000, 2000)...
[ 13343 ] Building cosine similarity matrix for [5277, 6277)...
[ 13344 ] Building cosine similarity matrix for [9554, 10554)...
[ 13345 ] Building cosine similarity matrix for [13831, 14831)...
[ 13342 ] Building cosine similarity matrix for [2000, 3000)...
[ 13343 ] Building cosine similarity matrix for [6277, 7277)...
[ 13344 ] Building cosine similarity matrix for [10554, 11554)...
[ 13345 ] Building cosine similarity matrix for [14831, 15831)...
[ 13342 ] Building cosine similarity matrix for [3000, 4000)...
[ 13345 ] Building cosine similarity matrix for [15831, 16831)...
[ 13344 ] Building cosine similarity matrix for [11554, 12554)...
[ 13343 ]

In [19]:
# Best params 0.53119031862654198, 0.8622528471283929, 0.449568357270693, 0.84725181341512967]
# [0.24995989924853496, 0.35681721890423695, 0.12389358733625891, 0.78110404084094665]
# weights = [0.53, 0.86, 0.45, 0.85]
# with MF [0.027625984433804714, 0.028305576119182392, 0.0054533687749983581, 0.17127788497293486, 0.022819397358672337]
# result is 0.12258991178466422 good!
# [0.21601720781243466, 0.16141373641762527, 0.30928573478985127, 0.72121294762063692, 0.037234416396506938]
S = S_icm * 0.21601720781243466 + S_urm * 0.16141373641762527 + S_user * 0.30928573478985127 + S_cbf_full * 0.72121294762063692 + S_mf * 0.037234416396506938 + S_icm2 * 0.02
print("Similarity matrix ready!")

# keep only topk?
S = top_k_filtering(S.transpose(), 300).transpose()
s_norm = S.sum(axis=0)
s_norm[s_norm==0] = 1
# Normalize S
S = S.multiply(csr_matrix(np.reciprocal(s_norm)))

# Keep only the target playlists in the URM
urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                   for x in tg_playlist]]

# Compute ratings
R_hat = urm_cleaned.dot(S.tocsc()).tocsr()
print("R_hat done")

# Remove the entries in R_hat that are already present in the URM
urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                              for x in tg_tracks]]
R_hat[urm_cleaned.nonzero()] = 0
R_hat.eliminate_zeros()

print("Shape of final matrix: ", R_hat.shape)
recs = predict(R_hat, list(tg_playlist), list(tg_tracks))
ev.evaluate_fold(recs)

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17351)
MAP@5: 0.1033928975344946


0.1033928975344946

In [100]:
def predict(R_hat, pl_id_list, tr_id_list, at=5):
        """
        returns a dictionary of
        'pl_id': ['tr_1', 'tr_at'] for each playlist in target playlist
        """
        recs = {}
        for i in range(0, R_hat.shape[0]):
            pl_id = pl_id_list[i]
            pl_row = R_hat.data[R_hat.indptr[i]:
                                     R_hat.indptr[i + 1]]
            # get top 5 indeces. argsort, flip and get first at-1 items
            sorted_row_idx = np.flip(pl_row.argsort(), axis=0)[0:at]
            track_cols = [R_hat.indices[R_hat.indptr[i] + x]
                          for x in sorted_row_idx]
            tracks_ids = [tr_id_list[x] for x in track_cols]
            recs[pl_id] = tracks_ids
        return recs

In [88]:
with open('submission_ensemble.csv', mode='w', newline='') as out:
    fieldnames = ['playlist_id', 'track_ids']
    writer = csv.DictWriter(out, fieldnames=fieldnames, delimiter=',')
    writer.writeheader()
    for k in tg_playlist:
        track_ids = ''
        for r in recs[k]:
            track_ids = track_ids + r + ' '
        writer.writerow({'playlist_id': k,
                         'track_ids': track_ids[:-1]})

# Bayesian opt

In [25]:
def mixS(params):
    global S_icm, S_user, S_urm, S_cbf_full, S_mf, urm, tg_playlist, evaluator
    print(params)

    S = S_icm * params[0] + S_urm * params[1] + S_user * params[2] + S_cbf_full * params[3] + S_mf * params[4]
    
    # maybe it's possible to avoid, who knows?
    S = top_k_filtering(S.transpose(), 300).transpose()
    
    s_norm = S.sum(axis=0)
    s_norm[s_norm==0] = 1
    # Normalize S
    S= S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")

    # Keep only the target playlists in the URM
    urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                       for x in tg_playlist]]

    # Compute ratings
    R_hat = urm_cleaned.dot(S.tocsc()).tocsr()
    print("R_hat done")

    # Remove the entries in R_hat that are already present in the URM
    urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                  for x in tg_tracks]]
    R_hat[urm_cleaned.nonzero()] = 0
    R_hat.eliminate_zeros()

    print("Shape of final matrix: ", R_hat.shape)
    recs = predict(R_hat, list(tg_playlist), list(tg_tracks))
    map_at_five = ev.evaluate_fold(recs)
    return -map_at_five

In [26]:
from skopt import forest_minimize
from skopt.space import Integer, Real
space = [Real(0.0, 1.0),  # ICM
         Real(0.0, 1.0),  # URM
         Real(0.0, 1.0),  # USER
         Real(0.0, 1.0),  # CBF_FULL
         Real(0.0, 1.0),  # MF
         Real(0.0, 1.0),  # ICM2
             ]
x0 = [1, 0, 0, 0, 0, 0]
x1 = [0, 1, 0, 0, 0, 0]
x2 = [0, 0, 1, 0, 0, 0]
x3 = [0, 0, 0, 1, 0, 0]
x4 = [0.53119031862654198, 0.8622528471283929, 0.449568357270693, 0.84725181341512967, 0, 0]
x5 = [0, 0, 0, 0, 1, 0]
x6 = [0.027625984433804714, 0.028305576119182392, 0.0054533687749983581, 0.17127788497293486, 0.022819397358672337, 0]
x0s = [x0, x1, x2, x3, x4, x5, x6]
res = forest_minimize(mixS, space, x0=x0s, verbose=True, n_random_starts=50, n_calls=1000, n_jobs=-1)
print('Maximimum p@k found: {:6.5f}'.format(-res.fun))
print('Optimal parameters:')
params = ['ICM', 'URM', 'USER', 'FULL']
for (p, x_) in zip(params, res.x):
    print('{}: {}'.format(p, x_))

Iteration No: 1 started. Evaluating function at provided point.
[1, 0, 0, 0, 0]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.10430596395988852
[0, 1, 0, 0, 0]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.08958531252356168
[0, 0, 1, 0, 0]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.07772751262911835
[0, 0, 0, 1, 0]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.12009500113096588
[0.531190318626542, 0.8622528471283929, 0.449568357270693, 0.8472518134151297, 0]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11996757897911488
[0, 0, 0, 0, 1]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.09148759707456817
[0.027625984433804714, 0.028305576119182392, 0.005453368774998358, 0.17127788497293486, 0.022819397358672337]
Similarity matrix ready!
R_hat done
Shape of final matrix:

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11881323984015693
Iteration No: 19 ended. Evaluation done at random point.
Time taken: 24.2869
Function value obtained: -0.1188
Current minimum: -0.1218
Iteration No: 20 started. Evaluating function at random point.
[0.099737562670298568, 0.20119381721619517, 0.08649259150128398, 0.12783417535646938, 0.622874316951467]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.1128259066576191
Iteration No: 20 ended. Evaluation done at random point.
Time taken: 24.2844
Function value obtained: -0.1128
Current minimum: -0.1218
Iteration No: 21 started. Evaluating function at random point.
[0.35152993543906991, 0.77717004161757663, 0.1330860317250421, 0.58971369793576267, 0.54841939396927608]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.1181135489708213
Iteration No: 21 ended. Evaluation done at random point.
Time taken: 23.9294
Function value o

Iteration No: 39 ended. Evaluation done at random point.
Time taken: 18.6184
Function value obtained: -0.1178
Current minimum: -0.1218
Iteration No: 40 started. Evaluating function at random point.
[0.87838315520772092, 0.038095952761533443, 0.46319594420160737, 0.73112199132675859, 0.1572122135007771]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11789941943753325
Iteration No: 40 ended. Evaluation done at random point.
Time taken: 18.7813
Function value obtained: -0.1179
Current minimum: -0.1218
Iteration No: 41 started. Evaluating function at random point.
[0.14371346654380038, 0.75758651073453387, 0.90763147087956653, 0.93903179593979402, 0.40036155640140558]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11743044560054293
Iteration No: 41 ended. Evaluation done at random point.
Time taken: 18.6169
Function value obtained: -0.1174
Current minimum: -0.1218
Iteration No: 42 started. Evaluating function at random

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11354218502601217
Iteration No: 60 ended. Search finished for the next optimal point.
Time taken: 19.4248
Function value obtained: -0.1135
Current minimum: -0.1218
Iteration No: 61 started. Searching for the next optimal point.
[5.3312886342693502e-05, 0.56429539691292352, 0.39967593185521366, 0.65341208350337732, 0.2833146112367822]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11451858553871672
Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 18.9452
Function value obtained: -0.1145
Current minimum: -0.1218
Iteration No: 62 started. Searching for the next optimal point.
[0.032711861220206757, 0.0022050858942574973, 0.13749147681652887, 0.0087612623373933846, 0.24179303668595747]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.10893538415139871
Iteration No: 62 ended. Search finished for the next optim

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.12065294428108272
Iteration No: 80 ended. Search finished for the next optimal point.
Time taken: 19.0731
Function value obtained: -0.1207
Current minimum: -0.1218
Iteration No: 81 started. Searching for the next optimal point.
[0.0005381545360371166, 0.10033415239014579, 0.96006445874473423, 0.51809962686256816, 0.02935415581853807]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11087913745004903
Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 19.0285
Function value obtained: -0.1109
Current minimum: -0.1218
Iteration No: 82 started. Searching for the next optimal point.
[0.052542161653094013, 0.93825887600488467, 0.28667955610020762, 0.17805222286288894, 0.0016272736535966064]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.10518057754655818
Iteration No: 82 ended. Search finished for the next optimal

...

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.10321345095378123
Iteration No: 900 ended. Search finished for the next optimal point.
Time taken: 18.8061
Function value obtained: -0.1032
Current minimum: -0.1218
Iteration No: 901 started. Searching for the next optimal point.
[0.11960660980230967, 0.6975987352675963, 0.48029711421886656, 0.0084012932081728459, 0.032636482864131573]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.10790997511875144
Iteration No: 901 ended. Search finished for the next optimal point.
Time taken: 18.7404
Function value obtained: -0.1079
Current minimum: -0.1218
Iteration No: 902 started. Searching for the next optimal point.
[0.057679494224529906, 0.09756262275960606, 0.53044167512111029, 0.0018127010546213911, 0.019345796305337285]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.1025265776973535
Iteration No: 902 ended. Search finished for the next op

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.0952522053833973
Iteration No: 920 ended. Search finished for the next optimal point.
Time taken: 18.9858
Function value obtained: -0.0953
Current minimum: -0.1218
Iteration No: 921 started. Searching for the next optimal point.
[0.98028774732607782, 0.97439558443234631, 0.33187483217350494, 6.7566329193802246e-05, 0.024990506106063355]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11679936665912692
Iteration No: 921 ended. Search finished for the next optimal point.
Time taken: 18.5841
Function value obtained: -0.1168
Current minimum: -0.1218
Iteration No: 922 started. Searching for the next optimal point.
[0.0035657040003121985, 0.0065668102076319474, 0.01708318963345901, 0.14048159812108943, 0.20405922360460643]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11346226343964397
Iteration No: 922 ended. Search finished for the next 

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11835557566161513
Iteration No: 940 ended. Search finished for the next optimal point.
Time taken: 18.6516
Function value obtained: -0.1184
Current minimum: -0.1218
Iteration No: 941 started. Searching for the next optimal point.
[0.35864231397454788, 0.76467187343379328, 0.83873081759632151, 0.00476512391263162, 0.38422066438700953]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11489632813089048
Iteration No: 941 ended. Search finished for the next optimal point.
Time taken: 18.7923
Function value obtained: -0.1149
Current minimum: -0.1218
Iteration No: 942 started. Searching for the next optimal point.
[0.00041630367153311376, 0.14329470096842226, 0.1647098927714469, 0.030823834141943902, 0.22354022440291493]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.10588026841589378
Iteration No: 942 ended. Search finished for the next opti

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11649928372163172
Iteration No: 960 ended. Search finished for the next optimal point.
Time taken: 18.8808
Function value obtained: -0.1165
Current minimum: -0.1218
Iteration No: 961 started. Searching for the next optimal point.
[0.0013459192041480608, 0.002763091503909521, 0.92828182012753224, 0.0039252054905165598, 0.077349290214155722]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.08394933273015139
Iteration No: 961 ended. Search finished for the next optimal point.
Time taken: 19.9557
Function value obtained: -0.0839
Current minimum: -0.1218
Iteration No: 962 started. Searching for the next optimal point.
[0.27471820471599834, 0.93389300864645353, 0.0044220127881760005, 0.3191924053180068, 0.33661275802081453]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11575284626404299
Iteration No: 962 ended. Search finished for the next 

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.1171876649325192
Iteration No: 980 ended. Search finished for the next optimal point.
Time taken: 18.7297
Function value obtained: -0.1172
Current minimum: -0.1218
Iteration No: 981 started. Searching for the next optimal point.
[0.0098343624633246991, 0.30804455853060914, 0.15565216327910195, 0.24919584142206824, 0.028351088727541114]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11328055492724119
Iteration No: 981 ended. Search finished for the next optimal point.
Time taken: 18.6727
Function value obtained: -0.1133
Current minimum: -0.1218
Iteration No: 982 started. Searching for the next optimal point.
[0.081237500678162836, 0.2124629193483572, 0.46379200498088158, 0.21019972914143653, 0.033138848201399458]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11527407072306425
Iteration No: 982 ended. Search finished for the next opti

# Ensemble user specific

Here we divide the number of ratings in n intervals and we find the correct mix of similarities for every interval

In [68]:
from skopt import forest_minimize
from skopt.space import Integer, Real

# best params [0.60743780798501301, 0.75766752585999741, 0.74643694551203255, 0.92217949239452435, 0.29563731859077508, 0.14148742619410806, 0.52523547719447961, 0.95025892687791746, 0.39335940969308369, 0.32999205674293214, 0.45124766380750581, 0.83841658583852363, 0.53465266748313023, 0.51635807787404442, 0.037395274863386237, 0.68357308283438822, 0.16711621806405755, 0.53400445992963852, 0.69157109724050636, 0.41306956719296062, 0.30457203528448956, 0.95820247202754694, 0.16658999330326466, 0.59519100134429148]

# number of intervals of number of ratings
intervals = 4
S_number = 6
space = [Real(0.0,1.0) for _ in range(S_number)]

space_all = []
for i in range(intervals):
    space_all = space_all + space
res = forest_minimize(mixSComplex, space_all, verbose=True, n_random_starts=100, n_calls=10000, n_jobs=-1)
print('Maximimum p@k found: {:6.5f}'.format(-res.fun))
print('Optimal parameters:')
params = ['ICM', 'URM', 'USER', 'FULL']
for (p, x_) in zip(params, res.x):
    print('{}: {}'.format(p, x_))

Iteration No: 1 started. Evaluating function at random point.
[0.24885532600321919, 0.33124286435142586, 0.30429957824989057, 0.29204634560478754, 0.70723267116860034, 0.77474538476242993, 0.22347708871068489, 0.12037641994528117, 0.25964027014882635, 0.27654989975400218, 0.97697681380577084, 0.26631333730925294, 0.64101379684944348, 0.02748368423816417, 0.31669562962344272, 0.67616621991375858, 0.78341827280282494, 0.27838965937740473, 0.06352134002875777, 0.48413708163331615, 0.49660366867432504, 0.052863307047735578, 0.44330598626900597, 0.49479643405807749]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11655130815049423
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 160.6006
Function value obtained: -0.1166
Current minimum: -0.1166
Iteration No: 2 started. Evaluating function at random point.
[0.77080785298819521, 0.67249768949973332, 0.65234183568879356

R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11778783080750997
Iteration No: 10 ended. Evaluation done at random point.
Time taken: 141.9318
Function value obtained: -0.1178
Current minimum: -0.1194
Iteration No: 11 started. Evaluating function at random point.
[0.66974303364508525, 0.99305969076784806, 5.4596252081040588e-05, 0.43030547813500764, 0.45619881870144663, 0.82149309400029802, 0.52399160525608623, 0.26624415348769731, 0.50726528493560219, 0.22618962437947893, 0.94142010755371297, 0.54456340596013308, 0.34172390819949722, 0.057135597668685603, 0.001284657158123981, 0.086545395665156746, 0.64987272638552185, 0.90665649850180419, 0.2624184674397983, 0.037473969936688174, 0.93551409900883742, 0.29617364472306823, 0.77239896572494005, 0.89815413184437276]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11798763477343002
Iteration No: 20 ended. Evaluation done at random point.
Time taken: 142.9742
Function value obtained: -0.1180
Current minimum: -0.1200
Iteration No: 21 started. Evaluating function at random point.
[0.01292734080448266, 0.7926282645908711, 0.14616433282390343, 0.88270807282841757, 0.7883247286723597, 0.5014596175358117, 0.47762874550491863, 0.8963223826382144, 0.60145363042852706, 0.37896977076737326, 0.11439617111958347, 0.59677336460779185, 0.29550470675480273, 0.1309347214909036, 0.77038761537294886, 0.096669565931990192, 0.11025500338993435, 0.74983143264401886, 0.61314925242741425, 0.7340805027144145, 0.86408712131082732, 0.52765498997783755, 0.40495422161603101, 0.60806080539638108]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matri

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11571816331146806
Iteration No: 30 ended. Evaluation done at random point.
Time taken: 141.3425
Function value obtained: -0.1157
Current minimum: -0.1200
Iteration No: 31 started. Evaluating function at random point.
[0.10805828676909204, 0.46819785709490591, 0.83721579177278493, 0.01626084571991682, 0.25595516651094213, 0.74851964857439834, 0.70371270911262873, 0.48182513646676606, 0.22473975836303378, 0.25331154825076635, 0.40544090202928518, 0.8091199313319114, 0.3845542336310338, 0.40856620357652207, 0.95366151374381192, 0.2861642292482301, 0.75169927439024142, 0.58037251565583958, 0.53592343385063845, 0.39888074077716307, 0.63441859451108695, 0.61027124508008757, 0.59705804005692131, 0.7075304099973494]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matr

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11979567217070068
Iteration No: 40 ended. Evaluation done at random point.
Time taken: 143.1076
Function value obtained: -0.1198
Current minimum: -0.1200
Iteration No: 41 started. Evaluating function at random point.
[0.1193172336031896, 0.68417267541242055, 0.71588995530586075, 0.58912507818653004, 0.85146794818073912, 0.36294628324962858, 0.55317736302716425, 0.85425525553866533, 0.96276589155876846, 0.12522399765124073, 0.066077211564006602, 0.19372982704288694, 0.44048252291280665, 0.62149992219999062, 0.92329594533979154, 0.88685373280903379, 0.63454578731019007, 0.67961636822738369, 0.78232278367458263, 0.77813096078797095, 0.54825092909653794, 0.93642202057176105, 0.82000952178711906, 0.2589438242881818]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final m

...

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.1169124632436104
Iteration No: 80 ended. Evaluation done at random point.
Time taken: 143.6481
Function value obtained: -0.1169
Current minimum: -0.1204
Iteration No: 81 started. Evaluating function at random point.
[0.56862741353787338, 0.083680033019733494, 0.0020874622408724179, 0.53785866198705845, 0.044811267054590383, 0.59302038637502785, 0.82209909400437109, 0.3733964302246508, 0.2107712037937402, 0.84296714168009901, 0.60579247140091774, 0.91100228097289171, 0.87295240816286734, 0.8315953238421917, 0.076912309537154791, 0.90974661147990954, 0.033596748666012817, 0.12915517972128401, 0.31479641574009481, 0.9356252966144929, 0.46991910127211178, 0.31580920113435668, 0.23827262791920373, 0.84867503343780859]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11915780743421578
Iteration No: 90 ended. Evaluation done at random point.
Time taken: 156.1766
Function value obtained: -0.1192
Current minimum: -0.1204
Iteration No: 91 started. Evaluating function at random point.
[0.48113818820047438, 0.064719419685571392, 0.31586069801410238, 0.60278733820804475, 0.87880487504526306, 0.69957550256590273, 0.85441085320153454, 0.70032286188703896, 0.977009270100341, 0.67063426157614137, 0.1798201455049763, 0.17830568792359971, 0.54244124444311936, 0.68490717145619506, 0.79670382679815843, 0.97085247993282919, 0.018388551834205517, 0.63618940389941592, 0.78205829676135374, 0.67433742915186823, 0.24809118408576386, 0.62308680576542563, 0.77345196089507651, 0.96504405147008987]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final m

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.1186669682575589
Iteration No: 100 ended. Evaluation done at random point.
Time taken: 146.4505
Function value obtained: -0.1187
Current minimum: -0.1204
Iteration No: 101 started. Searching for the next optimal point.
[0.022805100538983773, 0.74739084510155795, 0.9588051126351349, 0.91729340365757828, 0.15841790052650997, 0.027748635444192378, 0.27076693194508578, 0.21155884096711924, 0.3164429212594369, 0.59684786727727923, 0.63217296458474748, 0.93170594744664181, 0.49491265024929154, 0.49078777350977831, 0.80144049440588616, 0.74982827550037157, 0.2123846156738039, 0.30777259041235444, 0.77523921520706041, 0.31974343547330292, 0.029254583678066999, 0.96745511540113627, 0.39797816843005107, 0.89863702500286646]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of fina

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11673754052627616
Iteration No: 110 ended. Search finished for the next optimal point.
Time taken: 228.6684
Function value obtained: -0.1167
Current minimum: -0.1204
Iteration No: 111 started. Searching for the next optimal point.
[0.27838098821523954, 0.7975694647469509, 0.9445288899857085, 0.45601537009649307, 0.025361945405315563, 0.0048457505165416812, 0.04336124703855649, 0.5779529717162688, 0.038629196602532063, 0.8950952787499078, 0.032726711913735669, 0.30602111584368014, 0.80974905798221364, 0.30863838124743154, 0.26535860199041611, 0.076545781623942113, 0.0036686493965233389, 0.35828298062947356, 0.97863744152081933, 0.41513177801932188, 0.43711931060311049, 0.044875835297942708, 0.30838412291410527, 0.1042501909961541]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat do

KeyboardInterrupt: 

In [86]:
def mixSComplex(params):
    global S_icm, S_user, S_urm, S_cbf_full, S_mf, urm, tg_playlist, evaluator
    print(params)
    # list of matrices
    Ss = [S_icm, S_user, S_urm, S_cbf_full, S_mf, S_icm2]
    
    # this is a list of matrix mixed with theirs params
    Ss_mixed = []
    
    for i in range(intervals):
        # get the correct params
        params_ = params[i*S_number:(i+1)*S_number]
        S_cur = lil_matrix(Ss[0].shape)
        for j in range(len(Ss)):
            S_cur += params_[j] * Ss[j]
        # maybe it's possible to avoid, who knows?
        S_cur = top_k_filtering(S_cur.transpose(), 300).transpose()

        s_norm = S_cur.sum(axis=0)
        s_norm[s_norm==0] = 1
        # Normalize S
        S_cur = S_cur.multiply(csr_matrix(np.reciprocal(s_norm)))
        Ss_mixed.append(S_cur)
    
    # R_hat computations
    # Keep only the target playlists in the URM
    urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                       for x in tg_playlist]]
    # Remove the entries in R_hat that are already present in the URM
    urm_cleaned_t = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                    for x in tg_tracks]]
    R_hats = []
    for i in range(intervals):
        print(Ss_mixed[i].shape)
        # Compute ratings
        R_hat = urm_cleaned.dot(Ss_mixed[i].tocsc()).tocsr()
        print("R_hat done")

        
        R_hat[urm_cleaned_t.nonzero()] = 0
        R_hat.eliminate_zeros()
        R_hats.append(R_hat)
    
    # divide ratings in intervals
    n_ratings = np.array(urm.sum(axis=1)).squeeze()
    clusters = pd.cut(n_ratings, intervals, labels=False)
    clusters = clusters[[dataset.get_playlist_index_from_id(x)
                       for x in tg_playlist]]
    
    R_hat = lil_matrix(R_hats[i].shape)
    
    for i in range(intervals):
        R_hat[clusters==i] = R_hats[i][clusters==i]
    
    R_hat = csr_matrix(R_hat)
    print("Shape of final matrix: ", R_hat.shape)
    #recs = predict(R_hat, list(tg_playlist), list(tg_tracks))
    #map_at_five = ev.evaluate_fold(recs)
    #return -map_at_five
    return R_hat

In [104]:
def mixSCluster(params):
    global S_icm, S_user, S_urm, S_cbf_full, S_mf, urm, tg_playlist, evaluator, cluster_number, clusters
    print(params)
    S = S_icm * params[0] + S_urm * params[1] + S_user * params[2] + S_cbf_full * params[3] + S_mf * params[4] + S_icm2*params[5]
    
    # maybe it's possible to avoid, who knows?
    S = top_k_filtering(S.transpose(), 300).transpose()
    
    s_norm = S.sum(axis=0)
    s_norm[s_norm==0] = 1
    # Normalize S
    S= S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")

    # Keep only the target playlists in the URM
    urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                       for x in tg_playlist]]

    # Compute ratings
    R_hat = urm_cleaned.dot(S.tocsc()).tocsr()
    print("R_hat done")

    # Remove the entries in R_hat that are already present in the URM
    urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                  for x in tg_tracks]]
    R_hat[urm_cleaned.nonzero()] = 0
    R_hat.eliminate_zeros()

    recs = predict(R_hat, list(tg_playlist), list(tg_tracks))
    map_at_five = ev.evaluate_fold(recs)
    map_playlists = ev.get_map_playlists()
    maps = np.array([map_playlists[x] for x in list(tg_playlist)])
    maps = maps[clusters==cluster_number]
    return -np.mean(maps)

In [ ]:
from skopt import forest_minimize
from skopt.space import Integer, Real

# best params [0.60743780798501301, 0.75766752585999741, 0.74643694551203255, 0.92217949239452435, 0.29563731859077508, 0.14148742619410806, 0.52523547719447961, 0.95025892687791746, 0.39335940969308369, 0.32999205674293214, 0.45124766380750581, 0.83841658583852363, 0.53465266748313023, 0.51635807787404442, 0.037395274863386237, 0.68357308283438822, 0.16711621806405755, 0.53400445992963852, 0.69157109724050636, 0.41306956719296062, 0.30457203528448956, 0.95820247202754694, 0.16658999330326466, 0.59519100134429148]

# number of intervals of number of ratings
S_number = 6
space = [Real(0.0,1.0) for _ in range(S_number)]
             
x0 = [1, 0, 0, 0, 0, 0]
x1 = [0, 1, 0, 0, 0, 0]
x2 = [0, 0, 1, 0, 0, 0]
x3 = [0, 0, 0, 1, 0, 0]
x4 = [0.53119031862654198, 0.8622528471283929, 0.449568357270693, 0.84725181341512967, 0, 0]
x5 = [0, 0, 0, 0, 1, 0]
x6 = [0.027625984433804714, 0.028305576119182392, 0.0054533687749983581, 0.17127788497293486, 0.022819397358672337, 0]
x0s = [x0, x1, x2, x3, x4, x5, x6]
# divide ratings in intervals
intervals = 4
n_ratings = np.array(urm.sum(axis=1)).squeeze()
clusters = pd.cut(n_ratings, intervals, labels=False)
clusters = clusters[[dataset.get_playlist_index_from_id(x)
                   for x in tg_playlist]]
for i in range(intervals):
    cluster_number = i
    
    res = forest_minimize(mixSCluster, space, x0=x0s, verbose=True, n_random_starts=100, n_calls=1000, n_jobs=-1)
    print('Maximimum p@k found: {:6.5f}'.format(-res.fun))
    print('Optimal parameters for cluster :', i)
    params = ['ICM', 'URM', 'USER', 'FULL', 'MF', 'ICM2']
    for (p, x_) in zip(params, res.x):
        print('{}: {}'.format(p, x_))

Iteration No: 1 started. Evaluating function at provided point.
[1, 0, 0, 0, 0, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.10171228228907508
[0, 1, 0, 0, 0, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.0847847395008671
[0, 0, 1, 0, 0, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.07368996456306988
[0, 0, 0, 1, 0, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.11325265776973538
[0.531190318626542, 0.8622528471283929, 0.449568357270693, 0.8472518134151297, 0, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.1133529367413105
[0, 0, 0, 0, 1, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.08675563597979329
[0.027625984433804714, 0.028305576119182392, 0.005453368774998358, 0.17127788497293486, 0.022819397358672337, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.11374274296916233
Iteration No: 1 ended. Evaluation done at provided point.
Time taken: 137.0644
Function value obtained: -0.1218
Current minimum: -0.1218
Iteration No: 2 started. Evaluating function at provided point.
[0.

Similarity matrix ready!
R_hat done
MAP@5: 0.1098899193244365
Iteration No: 21 ended. Evaluation done at random point.
Time taken: 23.2674
Function value obtained: -0.1180
Current minimum: -0.1218
Iteration No: 22 started. Evaluating function at random point.
[0.65234201597318409, 0.64472192037651421, 0.84213462705453213, 0.46417302837196683, 0.95526114359075143, 0.65888924432072393]
Similarity matrix ready!
R_hat done
MAP@5: 0.1116218050214885
Iteration No: 22 ended. Evaluation done at random point.
Time taken: 21.3827
Function value obtained: -0.1190
Current minimum: -0.1218
Iteration No: 23 started. Evaluating function at random point.
[0.99184247108200918, 0.15473101417374371, 0.224249553439063, 0.69252779720829005, 0.79146091754178072, 0.8166540182219777]
Similarity matrix ready!
R_hat done
MAP@5: 0.10954308979868808
Iteration No: 23 ended. Evaluation done at random point.
Time taken: 19.3868
Function value obtained: -0.1158
Current minimum: -0.1218
Iteration No: 24 started. Evalu

Iteration No: 42 ended. Evaluation done at random point.
Time taken: 19.1852
Function value obtained: -0.1181
Current minimum: -0.1218
Iteration No: 43 started. Evaluating function at random point.
[0.8471574395952457, 0.24282125076510236, 0.15186022292550719, 0.37741676940875357, 0.97793001577083549, 0.37261088917076707]
Similarity matrix ready!
R_hat done
MAP@5: 0.10984392671341338
Iteration No: 43 ended. Evaluation done at random point.
Time taken: 18.8683
Function value obtained: -0.1170
Current minimum: -0.1218
Iteration No: 44 started. Evaluating function at random point.
[0.87450981658763893, 0.33982954376387825, 0.11373323851527098, 0.5777120756467139, 0.030151137610893171, 0.78433455600493418]
Similarity matrix ready!
R_hat done
MAP@5: 0.10894292392369764
Iteration No: 44 ended. Evaluation done at random point.
Time taken: 18.9152
Function value obtained: -0.1156
Current minimum: -0.1218
Iteration No: 45 started. Evaluating function at random point.
[0.43375052901969791, 0.015

Similarity matrix ready!
R_hat done
MAP@5: 0.11054060167382966
Iteration No: 64 ended. Evaluation done at random point.
Time taken: 18.8902
Function value obtained: -0.1181
Current minimum: -0.1218
Iteration No: 65 started. Evaluating function at random point.
[0.022722585973856905, 0.25817054443294668, 0.091869016916934179, 0.47341824193909954, 0.69908414727420509, 0.83694219719474217]
Similarity matrix ready!
R_hat done
MAP@5: 0.10966146422378051
Iteration No: 65 ended. Evaluation done at random point.
Time taken: 19.2782
Function value obtained: -0.1164
Current minimum: -0.1218
Iteration No: 66 started. Evaluating function at random point.
[0.99925198559198192, 0.45661179022663112, 0.77753298648747027, 0.19890389030956973, 0.6269403375174657, 0.84477793738026274]
Similarity matrix ready!
R_hat done
MAP@5: 0.10982583125989609
Iteration No: 66 ended. Evaluation done at random point.
Time taken: 19.3092
Function value obtained: -0.1161
Current minimum: -0.1218
Iteration No: 67 started.

Iteration No: 85 ended. Evaluation done at random point.
Time taken: 19.0987
Function value obtained: -0.1207
Current minimum: -0.1218
Iteration No: 86 started. Evaluating function at random point.
[0.45785690870277196, 0.93323672629224397, 0.40833239331822235, 0.0010013964309709691, 0.95080371000935748, 0.88344028817842024]
Similarity matrix ready!
R_hat done
MAP@5: 0.10975872728643611
Iteration No: 86 ended. Evaluation done at random point.
Time taken: 19.3094
Function value obtained: -0.1174
Current minimum: -0.1218
Iteration No: 87 started. Evaluating function at random point.
[0.80107020509773474, 0.37717376575317774, 0.48386461308110462, 0.911575507627438, 0.41580841532576096, 0.058135748567574211]
Similarity matrix ready!
R_hat done
MAP@5: 0.11336198446806915
Iteration No: 87 ended. Evaluation done at random point.
Time taken: 18.8276
Function value obtained: -0.1212
Current minimum: -0.1218
Iteration No: 88 started. Evaluating function at random point.
[0.78682389630658101, 0.0

...

Similarity matrix ready!
R_hat done
MAP@5: 0.10350901002789728
Iteration No: 618 ended. Search finished for the next optimal point.
Time taken: 19.5673
Function value obtained: -0.0601
Current minimum: -0.0839
Iteration No: 619 started. Searching for the next optimal point.
[0.20475376726788913, 0.04732578036365688, 0.13264729659136745, 0.19921573477897686, 0.0657806954505923, 0.015536735095985747]
Similarity matrix ready!
R_hat done
MAP@5: 0.11349996230113853
Iteration No: 619 ended. Search finished for the next optimal point.
Time taken: 19.3389
Function value obtained: -0.0805
Current minimum: -0.0839
Iteration No: 620 started. Searching for the next optimal point.
[0.0555994199338289, 0.019612967099284066, 0.72742723187525904, 0.10772877014839291, 0.021732337319325116, 0.19371061189952293]
Similarity matrix ready!
R_hat done
MAP@5: 0.10920153811354905
Iteration No: 620 ended. Search finished for the next optimal point.
Time taken: 19.5830
Function value obtained: -0.0735
Current mi

Similarity matrix ready!
R_hat done
MAP@5: 0.10928145969991707
Iteration No: 639 ended. Search finished for the next optimal point.
Time taken: 29.0983
Function value obtained: -0.0775
Current minimum: -0.0839
Iteration No: 640 started. Searching for the next optimal point.
[0.059724889131097206, 0.091545856308946419, 0.31604863564277358, 0.038903931426637291, 0.048230722683136787, 0.2429001337109204]
Similarity matrix ready!
R_hat done
MAP@5: 0.11102088516926806
Iteration No: 640 ended. Search finished for the next optimal point.
Time taken: 23.5221
Function value obtained: -0.0817
Current minimum: -0.0839
Iteration No: 641 started. Searching for the next optimal point.
[0.046390852509286547, 0.18091282994417593, 0.036401567558402743, 0.26228889748360545, 0.20358180616798954, 0.00010100351592179371]
Similarity matrix ready!
R_hat done
MAP@5: 0.11063560280479531
Iteration No: 641 ended. Search finished for the next optimal point.
Time taken: 22.2853
Function value obtained: -0.0700
Cur

Similarity matrix ready!
R_hat done
MAP@5: 0.11262761064615862
Iteration No: 660 ended. Search finished for the next optimal point.
Time taken: 30.9850
Function value obtained: -0.0800
Current minimum: -0.0839
Iteration No: 661 started. Searching for the next optimal point.
[0.040356226465570584, 0.23722288297063804, 0.030515508487046768, 0.12967091230148844, 0.038217518191404405, 0.0058648731462794829]
Similarity matrix ready!
R_hat done
MAP@5: 0.10910050516474405
Iteration No: 661 ended. Search finished for the next optimal point.
Time taken: 25.0845
Function value obtained: -0.0667
Current minimum: -0.0839
Iteration No: 662 started. Searching for the next optimal point.
[0.011960554716388908, 0.031490693739549054, 0.8229042378550534, 0.13408270078551135, 0.075179421670835259, 0.0046348909204751996]
Similarity matrix ready!
R_hat done
MAP@5: 0.09391314182311691
Iteration No: 662 ended. Search finished for the next optimal point.
Time taken: 22.8620
Function value obtained: -0.0569
Cu

Similarity matrix ready!
R_hat done
MAP@5: 0.1050772826660637
Iteration No: 681 ended. Search finished for the next optimal point.
Time taken: 19.8901
Function value obtained: -0.0703
Current minimum: -0.0839
Iteration No: 682 started. Searching for the next optimal point.
[0.124752667263649, 0.94286753076255259, 0.40948336337014024, 0.10901544221027695, 0.0096881116086271497, 0.42507899683135952]
Similarity matrix ready!
R_hat done
MAP@5: 0.11089421699464692
Iteration No: 682 ended. Search finished for the next optimal point.
Time taken: 20.0985
Function value obtained: -0.0734
Current minimum: -0.0839
Iteration No: 683 started. Searching for the next optimal point.
[0.070326089286298146, 0.0041565631735616124, 0.95227897345197066, 0.2043051483342892, 0.23905479350833275, 0.010659270112011511]
Similarity matrix ready!
R_hat done
MAP@5: 0.10220538339742154
Iteration No: 683 ended. Search finished for the next optimal point.
Time taken: 19.8677
Function value obtained: -0.0653
Current m

Similarity matrix ready!
R_hat done


Optimal parameters for cluster : 0
Maximimum p@k found: 0.12219
ICM: 0.009642311670812533
URM: 0.03549722742589868
USER: 0.045874651180874866
FULL: 0.13569872442995415
MF: 0.025973785644864862
ICM2: 0.01860721460454229

Optimal parameters for cluster : 1
Maximimum p@k found: 0.09962
ICM: 0.01845795814184859
URM: 0.11437368099762504
USER: 0.13630459070062287
FULL: 0.23946298281940864
MF: 0.006252094519587638
ICM2: 0.1630197073121197

In [87]:
params = [0.60743780798501301, 0.75766752585999741, 0.74643694551203255, 0.92217949239452435, 0.29563731859077508, 0.14148742619410806, 0.52523547719447961, 0.95025892687791746, 0.39335940969308369, 0.32999205674293214, 0.45124766380750581, 0.83841658583852363, 0.53465266748313023, 0.51635807787404442, 0.037395274863386237, 0.68357308283438822, 0.16711621806405755, 0.53400445992963852, 0.69157109724050636, 0.41306956719296062, 0.30457203528448956, 0.95820247202754694, 0.16658999330326466, 0.59519100134429148]

R_hat = mixSComplex(params)

[0.607437807985013, 0.7576675258599974, 0.7464369455120325, 0.9221794923945243, 0.2956373185907751, 0.14148742619410806, 0.5252354771944796, 0.9502589268779175, 0.3933594096930837, 0.32999205674293214, 0.4512476638075058, 0.8384165858385236, 0.5346526674831302, 0.5163580778740444, 0.03739527486338624, 0.6835730828343882, 0.16711621806405755, 0.5340044599296385, 0.6915710972405064, 0.4130695671929606, 0.30457203528448956, 0.9582024720275469, 0.16658999330326466, 0.5951910013442915]
(100000, 32195)
R_hat done
(100000, 32195)
R_hat done
(100000, 32195)
R_hat done
(100000, 32195)
R_hat done
Shape of final matrix:  (10000, 32195)


# ICM weight Optimization

In [27]:
# BEST PARAMS 0.080286755758088982, 0.18553973793824477, 0.0075749795138103666, 0.011466764550627563, 0.010195035817612831, 0.049304432078911345
def optIcm(params):
    global dataset, ev, tg_playlist, tg_tracks
    print(params)
    artist_weight, album_weight, duration_weight, playcount_weight, tag_weight, inferred_album_weight = params 
    dataset.set_track_attr_weights_2(artist_weight, album_weight, duration_weight, playcount_weight, tag_weight,
                                 0.0, inferred_album_weight, 0, 0.0)
    S = icm_sim2(dataset.build_icm(), dataset)
    # Keep only the target playlists in the URM
    urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                       for x in tg_playlist]]

    # Compute ratings
    R_hat = urm_cleaned.dot(S.tocsc()).tocsr()
    print("R_hat done")

    # Remove the entries in R_hat that are already present in the URM
    urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                  for x in tg_tracks]]
    R_hat[urm_cleaned.nonzero()] = 0
    R_hat.eliminate_zeros()

    print("Shape of final matrix: ", R_hat.shape)
    recs = predict(R_hat, list(tg_playlist), list(tg_tracks))
    map_at_five = ev.evaluate_fold(recs)
    return -map_at_five

In [28]:
from skopt import forest_minimize
from skopt.space import Integer, Real
space = [Real(0.0, 1.0),  # Artist
         Real(0.0, 1.0),  # Album
         Real(0.0, 1.0),  # Duration
         Real(0.0, 1.0),  # Playcount
         Real(0.0, 1.0),  # Tag
         Real(0.0, 1.0),  # Inf album
             ]
# 1.5, 1.6, 0.1, 0.2, 0.0, 0.0, 1.0, 0, 0.0 
x0 = [0.15, 0.16, 0.01, 0.02, 0, 1]
x0s = [x0]
res = forest_minimize(optIcm, space, x0=x0s, verbose=True, n_random_starts=100, n_calls=10000, n_jobs=-1)
print('Maximimum p@k found: {:6.5f}'.format(-res.fun))
print('Optimal parameters:')
params = ['Artist', 'Album', 'Duration', 'Playcount', 'Tag', 'Inf Album']
for (p, x_) in zip(params, res.x):
    print('{}: {}'.format(p, x_))

Iteration No: 1 started. Evaluating function at provided point.
[0.15, 0.16, 0.01, 0.02, 0, 1]
[ 1374 ] Building cosine similarity matrix for [0, 1000)...
[ 1375 ] Building cosine similarity matrix for [4337, 5337)...
[ 1376 ] Building cosine similarity matrix for [8674, 9674)...
Running 4 workers...
[ 1377 ] Building cosine similarity matrix for [13011, 14011)...
[ 1374 ] Building cosine similarity matrix for [1000, 2000)...
[ 1375 ] Building cosine similarity matrix for [5337, 6337)...
[ 1376 ] Building cosine similarity matrix for [9674, 10674)...
[ 1377 ] Building cosine similarity matrix for [14011, 15011)...
[ 1374 ] Building cosine similarity matrix for [2000, 3000)...
[ 1375 ] Building cosine similarity matrix for [6337, 7337)...
[ 1376 ] Building cosine similarity matrix for [10674, 11674)...
[ 1377 ] Building cosine similarity matrix for [15011, 16011)...
[ 1374 ] Building cosine similarity matrix for [3000, 4000)...
[ 1375 ] Building cosine similarity matrix for [7337, 8337)

[ 1411 ] Building cosine similarity matrix for [16011, 17011)...
[ 1408 ] Building cosine similarity matrix for [4000, 4337)...
[ 1409 ] Building cosine similarity matrix for [8337, 8674)...
[ 1410 ] Building cosine similarity matrix for [12674, 13011)...
[ 1411 ] Building cosine similarity matrix for [17011, 17348)...
[ 1408 ] Building cosine similarity matrix for [17348, 17351)...
S done
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17351)
MAP@5: 0.06339666742064357
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 80.3238
Function value obtained: -0.0634
Current minimum: -0.1025
Iteration No: 6 started. Evaluating function at random point.
[0.13217048676969492, 0.28656451869012939, 0.43178716185897215, 0.6421662438328527, 0.15099363933390778, 0.51653135925230975]
[ 1427 ] Building cosine similarity matrix for [0, 1000)...
Running 4 workers...
[ 1428 ] Building cosine similarity matrix for [4337, 5337)...
[ 1429 ] Building cosine similarity matr

[ 1466 ] Building cosine similarity matrix for [1000, 2000)...
[ 1467 ] Building cosine similarity matrix for [5337, 6337)...
[ 1468 ] Building cosine similarity matrix for [9674, 10674)...
[ 1469 ] Building cosine similarity matrix for [14011, 15011)...
[ 1466 ] Building cosine similarity matrix for [2000, 3000)...
[ 1467 ] Building cosine similarity matrix for [6337, 7337)...
[ 1468 ] Building cosine similarity matrix for [10674, 11674)...
[ 1469 ] Building cosine similarity matrix for [15011, 16011)...
[ 1466 ] Building cosine similarity matrix for [3000, 4000)...
[ 1467 ] Building cosine similarity matrix for [7337, 8337)...
[ 1468 ] Building cosine similarity matrix for [11674, 12674)...
[ 1469 ] Building cosine similarity matrix for [16011, 17011)...
[ 1466 ] Building cosine similarity matrix for [4000, 4337)...
[ 1467 ] Building cosine similarity matrix for [8337, 8674)...
[ 1468 ] Building cosine similarity matrix for [12674, 13011)...
[ 1469 ] Building cosine similarity matrix

[ 1511 ] Building cosine similarity matrix for [17348, 17351)...
S done
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17351)
MAP@5: 0.07534268265098379
Iteration No: 14 ended. Evaluation done at random point.
Time taken: 80.8355
Function value obtained: -0.0753
Current minimum: -0.1040
Iteration No: 15 started. Evaluating function at random point.
[0.47302284255060445, 0.74046748374952454, 0.50901895741738212, 0.05615774034736, 0.99670240279963873, 0.017405427274526434]
[ 1516 ] Building cosine similarity matrix for [0, 1000)...
Running 4 workers...
[ 1517 ] Building cosine similarity matrix for [4337, 5337)...
[ 1518 ] Building cosine similarity matrix for [8674, 9674)...
[ 1519 ] Building cosine similarity matrix for [13011, 14011)...
[ 1516 ] Building cosine similarity matrix for [1000, 2000)...
[ 1517 ] Building cosine similarity matrix for [5337, 6337)...
[ 1518 ] Building cosine similarity matrix for [9674, 10674)...
[ 1519 ] Building cosine similarity matrix

[ 1549 ] Building cosine similarity matrix for [2000, 3000)...
[ 1550 ] Building cosine similarity matrix for [6337, 7337)...
[ 1551 ] Building cosine similarity matrix for [10674, 11674)...
[ 1552 ] Building cosine similarity matrix for [15011, 16011)...
[ 1549 ] Building cosine similarity matrix for [3000, 4000)...
[ 1550 ] Building cosine similarity matrix for [7337, 8337)...
[ 1551 ] Building cosine similarity matrix for [11674, 12674)...
[ 1552 ] Building cosine similarity matrix for [16011, 17011)...
[ 1549 ] Building cosine similarity matrix for [4000, 4337)...
[ 1550 ] Building cosine similarity matrix for [8337, 8674)...
[ 1551 ] Building cosine similarity matrix for [12674, 13011)...
[ 1552 ] Building cosine similarity matrix for [17011, 17348)...
[ 1549 ] Building cosine similarity matrix for [17348, 17351)...
S done
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17351)
MAP@5: 0.08850109326698341
Iteration No: 19 ended. Evaluation done at random point.
Ti

...

[ 9743 ] Building cosine similarity matrix for [5337, 6337)...
[ 9744 ] Building cosine similarity matrix for [9674, 10674)...
[ 9745 ] Building cosine similarity matrix for [14011, 15011)...
[ 9742 ] Building cosine similarity matrix for [2000, 3000)...
[ 9743 ] Building cosine similarity matrix for [6337, 7337)...
[ 9744 ] Building cosine similarity matrix for [10674, 11674)...
[ 9745 ] Building cosine similarity matrix for [15011, 16011)...
[ 9743 ] Building cosine similarity matrix for [7337, 8337)...
[ 9742 ] Building cosine similarity matrix for [3000, 4000)...
[ 9744 ] Building cosine similarity matrix for [11674, 12674)...
[ 9745 ] Building cosine similarity matrix for [16011, 17011)...
[ 9743 ] Building cosine similarity matrix for [8337, 8674)...
[ 9744 ] Building cosine similarity matrix for [12674, 13011)...
[ 9742 ] Building cosine similarity matrix for [4000, 4337)...
[ 9745 ] Building cosine similarity matrix for [17011, 17348)...
[ 9743 ] Building cosine similarity matr

[ 9777 ] Building cosine similarity matrix for [17348, 17351)...
S done
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17351)
MAP@5: 0.08062957098695608
Iteration No: 921 ended. Search finished for the next optimal point.
Time taken: 85.9728
Function value obtained: -0.0806
Current minimum: -0.1051
Iteration No: 922 started. Searching for the next optimal point.
[0.30767378277190355, 0.16687641878876217, 0.20232091441395489, 0.8504719878260012, 0.075182589031791106, 0.89974714019807589]
[ 9781 ] Building cosine similarity matrix for [0, 1000)...
Running 4 workers...
[ 9782 ] Building cosine similarity matrix for [4337, 5337)...
[ 9783 ] Building cosine similarity matrix for [8674, 9674)...
[ 9784 ] Building cosine similarity matrix for [13011, 14011)...
[ 9781 ] Building cosine similarity matrix for [1000, 2000)...
[ 9782 ] Building cosine similarity matrix for [5337, 6337)...
[ 9783 ] Building cosine similarity matrix for [9674, 10674)...
[ 9784 ] Building cosine s

[ 9822 ] Building cosine similarity matrix for [14011, 15011)...
[ 9819 ] Building cosine similarity matrix for [2000, 3000)...
[ 9820 ] Building cosine similarity matrix for [6337, 7337)...
[ 9821 ] Building cosine similarity matrix for [10674, 11674)...
[ 9822 ] Building cosine similarity matrix for [15011, 16011)...
[ 9819 ] Building cosine similarity matrix for [3000, 4000)...
[ 9820 ] Building cosine similarity matrix for [7337, 8337)...
[ 9821 ] Building cosine similarity matrix for [11674, 12674)...
[ 9822 ] Building cosine similarity matrix for [16011, 17011)...
[ 9819 ] Building cosine similarity matrix for [4000, 4337)...
[ 9821 ] Building cosine similarity matrix for [12674, 13011)...
[ 9820 ] Building cosine similarity matrix for [8337, 8674)...
[ 9822 ] Building cosine similarity matrix for [17011, 17348)...
[ 9819 ] Building cosine similarity matrix for [17348, 17351)...
S done
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17351)
MAP@5: 0.091881927165

Process ForkPoolWorker-3845:
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/Users/emanueleghelfi/Development/RS/recsys_challenge_2017/src/utils/matrix_utils.py", line 244, in _work_compute_cosine
    S_den = S_num.copy()
  File "/Users/emanueleghelfi/Development/RS/lib/python3.6/site-packages/scipy/spa

KeyboardInterrupt: 